In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.image as mgimg
import matplotlib.colors as colors
import matplotlib.ticker as ticker
import scipy as sp
import numpy as np
import emcee
import testkit
import corner
import pickle as pickle
from IPython.display import display

import forwardmodel
import ciamod
import TPmod
import cloud
import band
import brewtools
import cloudpost
import pickup
from astropy.convolution import convolve, convolve_fft
from astropy.convolution import Gaussian1DKernel
from scipy.io.idl import readsav
from scipy import interpolate
from scipy.interpolate import interp1d
from scipy.interpolate import InterpolatedUnivariateSpline
from bensconv import prism_non_uniform
from bensconv import conv_uniform_R
from bensconv import conv_uniform_FWHM

In [ ]:
runname ="2M2224_ESlabSiO2SlabFeDeck"
# OK finish?
fin = 1
flatendchain, flatendprobs,ndim = brewtools.get_endchain(runname,fin)
theta_max_end = flatendchain[np.argmax(flatendprobs)]
max_end_like = np.amax(flatendprobs)
print(max_end_like)
samples = flatendchain


In [ ]:
print(theta_max_end)

In [ ]:
argfile =runname+"_runargs.pic"

runargs = brewtools.pickle_load(argfile)
#with open(argfile, 'rb') as input:
#    runargs = pickle.load(input) 

gases_myP,chemeq,dist, cloudtype,do_clouds,gasnum,cloudnum,inlinetemps,coarsePress,press,inwavenum,linelist,cia,ciatemps,use_disort,fwhm,obspec,proftype,do_fudge, prof,do_bff,bff_raw,ceTgrid,metscale,coscale = runargs



In [ ]:
logP = np.log10(press)
logcoarsePress = np.log10(coarsePress)
print(cloudtype.size)

In [ ]:
# get mu
allelse = 10**(-3.56) + 10**(-2.85) + 10**(-4.48) + 10**(-5.2)
fboth  = 1- allelse
fratio = 0.84
fH2 = fratio*fboth
fHe = (1.0  - fratio) * fboth
mu = (fH2*2.01588) + (fHe* 4.002602) + (10**(-3.56) * 18.01528) + (28.01040*10**(-2.85)) + (44.00000*10**(-4.48)) + (16.04276 *10**(-5.2))
R_spec = 1.38e-23 / (mu* 1.66e-27) 
print(mu)
print(R_spec)

In [ ]:
nlayers = len(press)
p1 = np.zeros_like(press)
p2 = np.zeros_like(press)
for i in range(0,nlayers):
    if (i == nlayers-1):
        p1[i] =  np.exp((0.5)*(np.log(press[i-1] * press[i])))
        p2[i] = (press[i]**2) / p1[i]
    else:
        p1[i] = np.exp(((1.5)*np.log(press[i])) - ((0.5)*np.log(press[i+1])))
        p2[i] = np.exp((0.5)*(np.log(press[i] * press[i+1])))



In [ ]:
import cloud_props
taucloud1_press,cloudmass,cloudnumdens = cloud_props.get(theta_max_end,chemeq,gasnum,fwhm,do_fudge,cloudtype,do_clouds,cloudnum,coarsePress,press,inwavenum)



In [ ]:
print(sum(cloudmass[0,:,0]))
print(sum(cloudmass[0,:,1]))
print(sum(cloudnumdens[0,:,0]))
print(sum(cloudnumdens[0,:,1]))



In [ ]:
ndz = abs((R_spec*press * 1e5 * np.log(p1/p2)) / (1.38e-23 * 10**(theta_max_end[9]-2)) ) # this is in / m2
cloud1_fract =  cloudnumdens[0,:,0] / (ndz[:]*1e-4)
null = np.where(cloud1_fract == 0)
cloud1_fract[null] = np.nan
print("max likelihood mean MgSiO3 fraction =",np.nanmean(np.log10(cloud1_fract)))

In [ ]:
ndz = abs((R_spec*press * 1e5 * np.log(p1/p2)) / (1.38e-23 * 10**(theta_max_end[9]-2)) ) # this is in / m2
cloud2_fract =  cloudnumdens[0,:,1] / (ndz[:]*1e-4)
null = np.where(cloud2_fract == 0)
cloud2_fract[null] = np.nan
print("max likelihood mean SiO2 fraction =",np.nanmean(np.log10(cloud2_fract)))

In [ ]:
MgSi = sum(cloudnumdens[0,:,0]) / (sum(cloudnumdens[0,:,0])+sum(cloudnumdens[0,:,1]))
print(MgSi)

In [ ]:
print(taucloud1_press.shape)
print(samples.shape)
print(sum(taucloud1_press[0,:,0]))

In [ ]:
#Tsamples = samples[:,ndim-5:]
nsamps = 50 #Tsamples.shape[0]
cloud1_mass = np.empty([nsamps])
cloud2_mass = np.empty([nsamps])
cloud1_num = np.empty([nsamps])
cloud2_num = np.empty([nsamps])
ndz = np.empty([nsamps,64])

cloud1_fract = np.empty([64,nsamps])
cloud2_fract = np.empty([64,nsamps])


samp= np.empty(ndim)

for i in range(0,nsamps):
    sid= np.random.randint(0,high = len(samples))
    samp = samples[sid,:]
    ndz[i,:] = abs(R_spec*press * 1e5 * np.log(p1/p2)) / (1.38e-23 * 10**(samp[9]-2)) 
    junk,mass,num = cloud_props.get(samp,chemeq,gasnum,fwhm,do_fudge,cloudtype,do_clouds,cloudnum,coarsePress,press,inwavenum)
    cloud1_mass[i] = sum(mass[0,:,0])
    cloud2_mass[i] = sum(mass[0,:,1])
    cloud1_num[i] = sum(num[0,:,0])
    cloud2_num[i] = sum(num[0,:,1])
    cloud1_fract[:,i] =  num[0,:,0] / (ndz[i,:] * 1e-4)
    cloud2_fract[:,i] =  num[0,:,1] / (ndz[i,:] * 1e-4)

    
MgSi = cloud1_num / (cloud1_num+cloud2_num)

In [ ]:
print("MgSiO3 column density = ",np.percentile(cloud1_mass, [16, 50, 84],axis=0))," g/cm2"
print("SiO2 column density = ",np.percentile(cloud2_mass, [16, 50, 84],axis=0))," g/cm2"
print("MgSiO3 column num density = ",np.percentile(cloud1_num, [16, 50, 84],axis=0))," /cm2"
print("SiO2 column num density = ",np.percentile(cloud2_num, [16, 50, 84],axis=0))," /cm2"
print("Mg/Si = ", np.percentile(MgSi,[16, 50, 84],axis=0))

In [ ]:
med_1_cd = 3.25385508e18
med_2_cd = 1.45058254e18
O_cond_cd = (2* med_2_cd) + (3*med_1_cd)
med_ndz = np.empty([64])
for i in range(0,64):
    med_ndz[i] = np.percentile(ndz[:,i]*1e-4,[50])

czone = np.where(np.log10(press) < -1.5)

O_gas_cd =  (10**(-3.56) + 10**(-2.85) + (2*(10**-4.48))) * np.sum(med_ndz[czone])

print(O_cond_cd /(O_gas_cd))

In [ ]:
plt.rc('font',family='Times New Roman')
fig=plt.figure(dpi=320)

plt.axis([-10.,0,2.3, -4])
plt.ylabel(r'log(P / bar)')
plt.xlabel('log frac')

plt.plot(np.log10(cf1_intervals[:,1]),np.log10(press),'r-')
plt.plot(np.log10(cf2_intervals[:,1]),np.log10(press), 'b-')

plt.plot([np.log10(mean1),np.log10(mean1)],[2.3,-4],'r--')
plt.plot([np.log10(mean2),np.log10(mean2)],[2.3,-4],'b--')


In [ ]:
# get diagnostics along with the spectrum
gnostics = 1
shiftspec, clphotspec, ophotspec,cfunc = testkit.modelspec(theta_max_end,runargs,gnostics)

wavenew = shiftspec[0,::-1]

nwave = inwavenum.size
cfunc = np.reshape(cfunc,[cfunc.shape[1],cfunc.shape[2]])
fwhm = 0.005
wlen = shiftspec.shape[1]
wint =  shiftspec[0,0] - shiftspec[0,wlen-1]
# convolve with instrumental profile
# start by setting up kernel
# First step is finding the array index length of the FWHM
disp = wint / wlen
gwidth = int((((fwhm / disp) // 2) * 2) +1)
# needs to be odd
# now get the kernel and convolve
gauss = Gaussian1DKernel(gwidth)

for ilayer in range (0,press.size):
    cfunc[:,ilayer] = convolve(cfunc[:,ilayer],gauss,boundary='extend')

tau1_cl_Press = convolve(clphotspec[0],gauss,boundary='extend')[::-1]
tau1_oth_Press = convolve(ophotspec[0],gauss,boundary='extend')[::-1]


temp = taucloud1_press[0,:,0][::-1]
wc1 = np.where(temp > 0.)
wave1 = wavenew[wc1]
wlen = len(wave1)
wint = wave1[wlen-1] - wave1[0]
disp = wint/wlen
gwidth =  int((((fwhm / disp) // 2) * 2) +1)
gauss = Gaussian1DKernel(gwidth)
#print(gwidth)
tau_cl1 = convolve(temp[wc1],gauss,boundary='extend')


temp = taucloud1_press[0,:,1][::-1]
#wc2 = np.where(temp > 0.)
wc2a = np.where(np.logical_and(temp > 0.,wavenew < 2.))
wc2b = np.where(np.logical_and(temp > 0.,wavenew > 5.))
wave2a = wavenew[wc2a]
wave2b = wavenew[wc2b]

wlen = len(wave2a)
wint = wave2a[wlen -1] - wave2a[0]
disp = wint/wlen
gwidth =  int((((fwhm / disp) // 2) * 2) +1)
gauss = Gaussian1DKernel(gwidth)
tau_cl2a = convolve(temp[wc2a],gauss,boundary='extend')

print(wave2b)
print(temp[wc2b])
wlen = len(wave2b)
wint = wave2b[wlen-1] - wave2b[0]
disp = wint/wlen
gwidth =  int((((fwhm / disp) // 2) * 2) +1)
gauss = Gaussian1DKernel(gwidth)
tau_cl2b = convolve(temp[wc2b],gauss,boundary='extend')


temp = taucloud1_press[0,:,2][::-1]
wc3 = np.where(temp > 0.)
wave3 = wavenew[wc3]
wlen = len(wave1)
wint = wave3[wlen-1] - wave3[0]
disp = wint/wlen
gwidth =  int((((fwhm / disp) // 2) * 2) +1)
gauss = Gaussian1DKernel(gwidth)
tau_cl3 = convolve(temp[wc3],gauss,boundary='extend')



press = press.reshape(64,)
normfunc = np.zeros_like(cfunc)
for iwave in range(0,nwave):
    totcont = np.sum(cfunc[iwave,:])
    normfunc[iwave,:] = cfunc[iwave,:] / totcont

    
plt.rc('font', family='serif')
plt.rc('text', usetex=False)
fig=plt.figure(dpi=120)
plt.axis([1.0,15.0,200,1e-4])

ax = fig.add_subplot(1,1,1)

ax.set_yscale('log')
ax.set_xscale('log')
#major_ticks = np.arange(1.0,15.,1.0)
#minor_ticks = np.arange(1.0,15.,0.5)
#ax.set_xticks(major_ticks)                                                       
#ax.set_xticks(minor_ticks, minor=True)                                           

plt.pcolormesh(wavenew,press,(normfunc[::-1,:].transpose()),cmap='Greys',norm=colors.SymLogNorm(linthresh=0.001,linscale=0.00001,
                                              vmin=0., vmax=np.amax(normfunc)))

#t1, = plt.plot(wavenew,(tau1_cl_Press),'m-',label=r'$\tau_{clouds} = 1.0$')
t2, = plt.plot(wavenew,(tau1_oth_Press),'c-', label =r'$\tau_{gas} = 1.0$')



t3, = plt.plot(wave1,tau_cl1,'m--',label=r'$\tau_{MgSiO_3} = 1.0$')
t4, = plt.plot(wave2a,tau_cl2a,c='orange',linestyle='--',label=r'$\tau_{SiO_2} = 1.0$')
t4a, = plt.plot(wave2b,tau_cl2b,c='orange',linestyle='--')
t5, = plt.plot(wave3,tau_cl3,'r--', label =r'$\tau_{Fe} = 1.0$')



plt.legend(handles=[t2,t3,t4,t5],bbox_to_anchor=(0., 1.02, 1., .102), loc='lower left',
           ncol=2, mode="expand", borderaxespad=0.)

locs = np.append(np.arange(1.0,3.0,0.5),np.arange(3.0,6.0,1.0))
locs = np.append(locs,np.arange(6.0,10.,2.0))
locs = np.append(locs,np.arange(10.0,15.,4.0))

ax.xaxis.set_minor_locator(ticker.FixedLocator(locs))
ax.xaxis.set_major_locator(ticker.NullLocator())

ax.xaxis.set_minor_formatter(ticker.ScalarFormatter())



cbar = plt.colorbar(orientation='vertical',norm=colors.Normalize(clip=False),ticks=[1e-3,1e-2,0.1])
cbar.ax.set_yticklabels(['<0.1%', '1%', '10%'])
cbar.set_label('% of total', rotation=270)
plt.ylabel('Pressure / bar',fontsize=10)
plt.xlabel(r'Wavelength / $\mu m$',fontsize=10)
plt.savefig(runname+'_cont_with_CloudIDs.png',bbox_inches='tight',format='png', dpi=320)